In [1]:
import numpy as np
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

np.random.seed(42)
random.seed(42)

from DataExploration import DataExploration
from assignment1.DecisionTree import DecisionTree
from assignment1.NeuralNetwork import DNN
from assignment1.Boosting import Boosting
from assignment1.KNN import KNN
from assignment1.NeuralNetworkKeras import NN_Keras
from assignment1.SVM import SVM

from utils import make_timing_curve


Using TensorFlow backend.


In [3]:
# get datasets ready
de = DataExploration()

twitter_df = de.prepare_twitter_df(rewrite=True)
speed_dating_df = de.prepare_speed_dating_df()
encoded_speed_dating_df, labelled_speed_dating_df, label_encoder = de.get_hot_encoded_speed_dating(speed_dating_df)

(22223, 10)


(19276, 10)


(41499, 7)


(8378, 44)
removing empty values shape:
(6794, 44)


C:\Users\matan\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\matan\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you us

In [4]:
#encoded_speed_dating_df['decision'] = np.where(encoded_speed_dating_df['decision'] == 0, [-1, 1])
print(encoded_speed_dating_df['decision'].head(5))
print(np.sum(encoded_speed_dating_df['decision']))

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
Name: decision, dtype: float64
-756.0


In [8]:
X_train, X_test, y_train, y_test = de.get_train_test_validation(encoded_speed_dating_df, 'speed_dating')

In [4]:
# de.investigate_twitter(twitter_df)
# de.investigate_speed_dating(labelled_speed_dating_df, label_encoder)

# Speed Dating Learners


In [4]:
#best_params = {'dt__alpha': -0.01, 'dt__max_depth': 9, 'dt__min_samples_leaf': 7} #twitter
best_params = {'dt__alpha': 0.0001, 'dt__max_depth': 9, 'dt__min_samples_leaf': 4} #speed_dating  (CV score=0.761):
dt_clf = DecisionTree(model_name='dt_pruning_1', dataset_name='speed_dating', best_params=best_params)
dt_clf.train(X_train, y_train)
y_pred = dt_clf.predict(X_test)
dt_clf.evaluate_model(y_test=y_test, y_pred=y_pred)

#print(dt_clfr.get_model().best_estimator_.named_steps['dt'].tree_.max_depth)


[[527 107]
 [158 304]]
             precision    recall  f1-score   support

        0.0       0.77      0.83      0.80       634
        1.0       0.74      0.66      0.70       462

avg / total       0.76      0.76      0.76      1096



In [5]:
dt_best_params = dt_clf.get_best_params()
print(dt_best_params)
dt_clf_model = dt_clf.get_model()
#print(dt_clf.feature_importance_)

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, dt_clf_model, 'dt_pruning_1', 'speed_dating',
                              'DT')


{'dt__alpha': 0.0001, 'dt__max_depth': 9, 'dt__min_samples_leaf': 4}


dt_pruning_1 speed_dating 0.1


dt_pruning_1 speed_dating 0.2


dt_pruning_1 speed_dating 0.3


dt_pruning_1 speed_dating 0.4


dt_pruning_1 speed_dating 0.5


dt_pruning_1 speed_dating 0.6


dt_pruning_1 speed_dating 0.7


dt_pruning_1 speed_dating 0.8


dt_pruning_1 speed_dating 0.9


In [7]:
# 
# def iteration_lc(clf, X_train, y_train, X_test, y_test, params, clf_type=None, dataset=None,
#                  dataset_readable_name=None, balanced_dataset=False, x_scale='linear', seed=42, threads=1):
#     logger.info("Building iteration learning curve for params {} ({} threads)".format(params, threads))
# 
#     if clf_type is None or dataset is None:
#         raise Exception('clf_type and dataset are required')
#     if seed is not None:
#         np.random.seed(seed)
# 
#     curr_scorer = scorer
#     acc_method = balanced_accuracy
#     if not balanced_dataset:
#         curr_scorer = f1_scorer
#         acc_method = f1_accuracy
# 
#     cv = ms.GridSearchCV(clf, n_jobs=threads, param_grid=params, refit=True, verbose=10, cv=5, scoring=curr_scorer)
#     cv.fit(training_x, training_y)
#     reg_table = pd.DataFrame(cv.cv_results_)
#     reg_table.to_csv('{}/ITER_base_{}_{}.csv'.format(OUTPUT_DIRECTORY, clf_type, dataset), index=False)
#     d = defaultdict(list)
#     name = list(params.keys())[0]
#     for value in list(params.values())[0]:
#         d['param_{}'.format(name)].append(value)
#         clf.set_params(**{name: value})
#         clf.fit(training_x, training_y)
#         pred = clf.predict(training_x)
#         d['train acc'].append(acc_method(training_y, pred))
#         clf.fit(training_x, training_y)
#         pred = clf.predict(test_x)
#         d['test acc'].append(acc_method(test_y, pred))
#         logger.info(' - {}'.format(value))
#     d = pd.DataFrame(d)
#     d.to_csv('{}/ITERtestSET_{}_{}.csv'.format(OUTPUT_DIRECTORY, clf_type, dataset), index=False)
#     plt = plot_learning_curve('{} - {} ({})'.format(clf_type, dataset_readable_name, name),
#                               d['param_{}'.format(name)], d['train acc'], d['test acc'],
#                               multiple_runs=False, x_scale=x_scale,
#                               x_label='Value')
#     plt.savefig('{}/images/{}_{}_ITER_LC.png'.format(OUTPUT_DIRECTORY, clf_type, dataset), format='png', dpi=150)
# 
#     logger.info(" - Iteration learning curve complete")
# 
#     return cv

In [8]:
#best_params = {'boost__base_estimator__max_depth': 12, 'boost__n_estimators': 80}
best_params = {'boost__base_estimator__alpha': 0.01, 'boost__base_estimator__max_depth': 33, 'boost__n_estimators': 40} # (CV score=0.804) speed_dating:
boosting_clf = Boosting(model_name='boosting_1', dataset_name='speed_dating', best_params=best_params)
boosting_clf.train(X_train, y_train)
y_pred = boosting_clf.predict(X_test)
boosting_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


[[506 128]
 [120 342]]
             precision    recall  f1-score   support

        0.0       0.81      0.80      0.80       634
        1.0       0.73      0.74      0.73       462

avg / total       0.77      0.77      0.77      1096



In [9]:
boosting_best_params = boosting_clf.get_best_params()
print(boosting_best_params)
boosting_clf_model = boosting_clf.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, boosting_clf_model, 'boosting_1', 'speed_dating',
                              'Adaboost Classifier')

{'boost__base_estimator__alpha': 0.01, 'boost__base_estimator__max_depth': 33, 'boost__n_estimators': 40}


boosting_1 speed_dating 0.1


boosting_1 speed_dating 0.2


boosting_1 speed_dating 0.3


boosting_1 speed_dating 0.4


boosting_1 speed_dating 0.5


boosting_1 speed_dating 0.6


boosting_1 speed_dating 0.7


boosting_1 speed_dating 0.8


boosting_1 speed_dating 0.9


In [10]:
#best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'} #twitter
best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 121, 'knn__weights': 'distance'} # (CV score=0.749): #speeddating
knn_clf = KNN(model_name='knn_1', dataset_name='speed_dating', best_params=best_params)
knn_clf.train(X_train, y_train)
y_pred = knn_clf.predict(X_test)
knn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


[[562  72]
 [189 273]]
             precision    recall  f1-score   support

        0.0       0.75      0.89      0.81       634
        1.0       0.79      0.59      0.68       462

avg / total       0.77      0.76      0.75      1096



In [11]:
knn_best_params = knn_clf.get_best_params()
print(knn_best_params)
knn_clf_model = knn_clf.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, knn_clf_model, 'knn_1', 'speed_dating',
                              'k-NN')

{'knn__metric': 'manhattan', 'knn__n_neighbors': 121, 'knn__weights': 'distance'}


knn_1 speed_dating 0.1


knn_1 speed_dating 0.2


knn_1 speed_dating 0.3


knn_1 speed_dating 0.4


knn_1 speed_dating 0.5


knn_1 speed_dating 0.6


knn_1 speed_dating 0.7


knn_1 speed_dating 0.8


knn_1 speed_dating 0.9


In [4]:


#best_params = {'batch_size': 10, 'epochs': 1, 'layer_sizes': [12, 1], 'learning_rate': 0.0005,
#               'loss': 'mean_squared_error'}
#best_params = {'dnn__batch_size': 100, 'dnn__epochs': 50, 'dnn__layer_sizes': [180, 50], 'dnn__learning_rate': 0.0005, 'dnn__loss': 'mean_squared_error'} #
#best_params = {'dnn__batch_size': 100, 'dnn__epochs': 3, 'dnn__layer_sizes': [250, 50], 'dnn__learning_rate': 0.05, 'dnn__loss': 'binary_crossentropy'} #
keras_nn_clf = NN_Keras(model_name='keras_nn_1', dataset_name='speed_dating') #, best_params=best_params)
keras_nn_clf.train(X_train, y_train)
y_pred = keras_nn_clf.predict(X_test)
keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)



Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  7.9min


[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed: 558.3min


[Parallel(n_jobs=3)]: Done 240 out of 240 | elapsed: 674.2min finished


Best: 0.608170 using {'dnn__batch_size': 100, 'dnn__epochs': 500, 'dnn__layer_sizes': [180, 50], 'dnn__learning_rate': 0.001, 'dnn__loss': 'binary_crossentropy'}
0.582154 (0.035209) with: {'dnn__batch_size': 10, 'dnn__epochs': 20, 'dnn__layer_sizes': [180, 50], 'dnn__learning_rate': 0.001, 'dnn__loss': 'binary_crossentropy'}
0.557280 (0.084166) with: {'dnn__batch_size': 10, 'dnn__epochs': 20, 'dnn__layer_sizes': [180, 50], 'dnn__learning_rate': 0.001, 'dnn__loss': 'mean_squared_error'}
0.543131 (0.051752) with: {'dnn__batch_size': 10, 'dnn__epochs': 20, 'dnn__layer_sizes': [250, 80], 'dnn__learning_rate': 0.001, 'dnn__loss': 'binary_crossentropy'}
0.508672 (0.066808) with: {'dnn__batch_size': 10, 'dnn__epochs': 20, 'dnn__layer_sizes': [250, 80], 'dnn__learning_rate': 0.001, 'dnn__loss': 'mean_squared_error'}
0.520310 (0.064234) with: {'dnn__batch_size': 10, 'dnn__epochs': 20, 'dnn__layer_sizes': [320, 60], 'dnn__learning_rate': 0.001, 'dnn__loss': 'binary_crossentropy'}
0.485167 (0.065

...

C:\Users\matan\Anaconda3\lib\site-packages\sklearn\base.py:115: DeprecationWarning: Estimator KerasClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  else:


C:\Users\matan\Anaconda3\lib\site-packages\sklearn\base.py:115: DeprecationWarning: Estimator KerasClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  else:


C:\Users\matan\Anaconda3\lib\site-packages\sklearn\base.py:115: DeprecationWarning: Estimator KerasClassifier modifies parameters in __init__. This behavior is deprecated as of 0.18 and support for this behavior will be removed in 0.20.
  else:


[[  0 634]
 [  0 462]]
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00       634
        1.0       0.42      1.00      0.59       462

avg / total       0.18      0.42      0.25      1096



C:\Users\matan\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  


In [5]:
keras_nn_best_params = keras_nn_clf.get_best_params()
print(keras_nn_best_params)

keras_nn_clf_model = keras_nn_clf.get_model()

# timing_df = make_timing_curve(X_train, y_train, X_test, y_test, keras_nn_clf_model, 'keras_nn_1', 'speed_dating',
#                               'Feed-Forward Neural Network')

{'dnn__batch_size': 100, 'dnn__epochs': 500, 'dnn__layer_sizes': [180, 50], 'dnn__learning_rate': 0.001, 'dnn__loss': 'binary_crossentropy'}


In [14]:
#best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
best_params = {'svm__C': 2, 'svm__gamma': 0.00390625, 'svm__kernel': 'rbf'} # (CV score=0.781)
svm_clf_rbf = SVM(model_name='svm_rbf_1', dataset_name='speed_dating', best_params=best_params)
svm_clf_rbf.train(X_train, y_train,'rbf')
y_pred = svm_clf_rbf.predict(X_test)
svm_clf_rbf.evaluate_model(y_test=y_test, y_pred=y_pred)


[[527 107]
 [144 318]]
             precision    recall  f1-score   support

        0.0       0.79      0.83      0.81       634
        1.0       0.75      0.69      0.72       462

avg / total       0.77      0.77      0.77      1096



In [ ]:
y_train.describe()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\matan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2963, in run_code
    user's matplotlibrc configuration file).  Note that not all backends
  File "<ipython-input-9-c5d653196efe>", line 1, in <module>
    y.describe()
NameError: name 'y' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\matan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 1863, in showtraceback
    self.showsyntaxerror()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\matan\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1095, in get_records
  File "C:\Users\matan\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 311, in wrapped
    inspect.getargs = getargs
  File "C:\User

NameError: name 'y' is not defined

In [6]:
svm_best_params = svm_clf_rbf.get_best_params()
print(svm_best_params) 
svm_clf_rbf_model = svm_clf_rbf.get_model()
#{'svm__degree': 1, 'svm__kernel': 'rbf'}
# (CV score=0.781):
# {'svm__C': 16.0, 'svm__gamma': 0.00390625, 'svm__kernel': 'rbf'}

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, svm_clf_rbf_model, 'svm_rbf_1', 'speed_dating',
                              'SVM (RBF)')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\matan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2963, in run_code
    user's matplotlibrc configuration file).  Note that not all backends
  File "<ipython-input-6-f3dff103b964>", line 1, in <module>
    svm_best_params = svm_clf_rbf.get_best_params()
NameError: name 'svm_clf_rbf' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\matan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 1863, in showtraceback
    self.showsyntaxerror()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\matan\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1095, in get_records
  File "C:\Users\matan\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 311, in wrapped


NameError: name 'svm_clf_rbf' is not defined

In [16]:
# #best_params = {'svm__C': 0.25, 'svm__degree': 1, 'svm__gamma': 0.25, 'svm__kernel': 'rbf'}
# best_params = {'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly', 'svm__degree':19} #r (CV score=0.764): 
# 
# svm_clf_poly = SVM(model_name='svm_poly_1', dataset_name='speed_dating', best_params=best_params)
# svm_clf_poly.train(X_train, y_train,'poly')
# y_pred = svm_clf_poly.predict(X_test)
# svm_clf_poly.evaluate_model(y_test=y_test, y_pred=y_pred)


In [18]:
# svm_best_params = svm_clf_poly.get_best_params()
# print(svm_best_params) 
# svm_clf_poly_model = svm_clf_poly.get_model()
# 
# timing_df = make_timing_curve(X_train, y_train, X_test, y_test, svm_clf_poly_model, 'svm_poly_1', 'speed_dating',
#                               'SVM (Poly)')

{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'}


In [8]:
#Best parameter (CV score=0.784):
best_params = {'svm__C': 0.03125, 'svm__gamma': 0.001953125, 'svm__kernel': 'linear'}

svm_clf_linear_sigmoid = SVM(model_name='svm_linear_sigmoid_1', dataset_name='speed_dating', best_params=best_params)
svm_clf_linear_sigmoid.train(X_train, y_train,'linear_sigmoid')
y_pred = svm_clf_linear_sigmoid.predict(X_test)
svm_clf_linear_sigmoid.evaluate_model(y_test=y_test, y_pred=y_pred)


[[530 104]
 [132 330]]
             precision    recall  f1-score   support

        0.0       0.80      0.84      0.82       634
        1.0       0.76      0.71      0.74       462

avg / total       0.78      0.78      0.78      1096



In [10]:
svm_best_params = svm_clf_linear_sigmoid.get_best_params()
print(svm_best_params) 
svm_clf_linear_sigmoid_model = svm_clf_linear_sigmoid.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, svm_clf_linear_sigmoid_model, 'svm_sigmoid_linear_1', 'speed_dating',
                              'SVM (Linear)')

{'svm__C': 0.03125, 'svm__gamma': 0.001953125, 'svm__kernel': 'linear'}


svm_sigmoid_linear_1 speed_dating 0.1


svm_sigmoid_linear_1 speed_dating 0.2


svm_sigmoid_linear_1 speed_dating 0.3


svm_sigmoid_linear_1 speed_dating 0.4


svm_sigmoid_linear_1 speed_dating 0.5


svm_sigmoid_linear_1 speed_dating 0.6


svm_sigmoid_linear_1 speed_dating 0.7


svm_sigmoid_linear_1 speed_dating 0.8


svm_sigmoid_linear_1 speed_dating 0.9


# Twitter Learners

In [4]:
X_train, X_test, y_train, y_test = de.get_train_test_validation(twitter_df, 'twitter')


In [12]:
best_params = {'dt__alpha': 0.0001, 'dt__max_depth': 17, 'dt__min_samples_leaf': 9} #twitter(CV score=0.912)
dt_clf_t = DecisionTree(model_name='dt_pruning_1', dataset_name='twitter', best_params=best_params)
dt_clf_t.train(X_train, y_train)
y_pred = dt_clf_t.predict(X_test)
dt_clf_t.evaluate_model(y_test=y_test, y_pred=y_pred)

#print(dt_clfr.get_model().best_estimator_.named_steps['dt'].tree_.max_depth)


[[3494  388]
 [ 292 4126]]
             precision    recall  f1-score   support

         -1       0.92      0.90      0.91      3882
          1       0.91      0.93      0.92      4418

avg / total       0.92      0.92      0.92      8300



In [13]:
dt_best_params = dt_clf_t.get_best_params()
print(dt_best_params)
dt_clf_t_model = dt_clf_t.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, dt_clf_t_model, 'dt_pruning_1', 'twitter',
                              'DT')


{'dt__alpha': 0.0001, 'dt__max_depth': 17, 'dt__min_samples_leaf': 9}


dt_pruning_1 twitter 0.1


dt_pruning_1 twitter 0.2


dt_pruning_1 twitter 0.3


dt_pruning_1 twitter 0.4


dt_pruning_1 twitter 0.5


dt_pruning_1 twitter 0.6


dt_pruning_1 twitter 0.7


dt_pruning_1 twitter 0.8


dt_pruning_1 twitter 0.9


In [15]:
best_params = {'boost__base_estimator__alpha': 0.01, 'boost__base_estimator__max_depth': 25, 'boost__n_estimators': 40} #(CV score=0.918):
boosting_clf_t = Boosting(model_name='boosting_1', dataset_name='twitter', best_params=best_params)
boosting_clf_t.train(X_train, y_train)
y_pred = boosting_clf_t.predict(X_test)
boosting_clf_t.evaluate_model(y_test=y_test, y_pred=y_pred)


KeyboardInterrupt: 

In [ ]:
boosting_best_params = boosting_clf_t.get_best_params()
print(boosting_best_params)
boosting_clf_t_model = boosting_clf.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, boosting_clf_t_model, 'boosting_1', 'twitter',
                              'Adaboost Classifier')


None


In [16]:
best_params = {'knn__metric': 'manhattan', 'knn__n_neighbors': 10, 'knn__weights': 'distance'} #twitter
#{'knn__metric': 'manhattan', 'knn__n_neighbors': 49, 'knn__weights': 'distance'} (CV score=0.728): #speeddating
knn_clf_t = KNN(model_name='knn_1', dataset_name='twitter', best_params=best_params)
knn_clf_t.train(X_train, y_train)
y_pred = knn_clf_t.predict(X_test)
knn_clf_t.evaluate_model(y_test=y_test, y_pred=y_pred)


[[3453  429]
 [ 531 3887]]
             precision    recall  f1-score   support

         -1       0.87      0.89      0.88      3882
          1       0.90      0.88      0.89      4418

avg / total       0.88      0.88      0.88      8300



In [ ]:
knn_best_params = knn_clf_t.get_best_params()
print(knn_best_params)
knn_clf_t_model = knn_clf_t.get_model()


# timing_df = make_timing_curve(X_train, y_train, X_test, y_test, knn_clf_t_model, 'knn_1', 'twitter',
#                               'k-NN')

In [18]:
best_params = {'svm__C': 2.0, 'svm__gamma': 0.015625, 'svm__kernel': 'rbf'}
svm_clf_rbf_t = SVM(model_name='svm_rbf_1', dataset_name='twitter', best_params=best_params)
svm_clf_rbf_t.train(X_train, y_train,'rbf')
y_pred = svm_clf_rbf_t.predict(X_test)
svm_clf_rbf_t.evaluate_model(y_test=y_test, y_pred=y_pred)

In [ ]:
svm_best_params = svm_clf_rbf_t.get_best_params()
print(svm_best_params) 
svm_clf_rbf_t_model = svm_clf_rbf_t.get_model()

# timing_df = make_timing_curve(X_train, y_train, X_test, y_test, svm_clf_rbf_t_model, 'svm_rbf_1', 'twitter',
#                               'SVM (RBF)')
#Best parameter (CV score=0.893):
#{'svm__C': 16.0, 'svm__gamma': 1.0, 'svm__kernel': 'rbf'}

NameError: name 'svm_clf_rbf_t' is not defined

In [ ]:
# #best_params = 
# #{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'} r (CV score=0.764):
# 
# svm_clf_poly_t = SVM(model_name='svm_poly_1', dataset_name='twitter')
# svm_clf_poly_t.train(X_train, y_train,'poly')
# y_pred = svm_clf_poly_t.predict(X_test)
# svm_clf_poly_t.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
# svm_best_params = svm_clf_poly.get_best_params()
# print(svm_best_params) 
# svm_clf_poly_t_model = svm_clf_poly_t.get_model()
# #{'svm__degree': 1, 'svm__kernel': 'rbf'}

In [ ]:
best_params = {'svm__C': 2.0, 'svm__gamma': 0.001953125, 'svm__kernel': 'linear'}
# #{'svm__C': 0.0625, 'svm__gamma': 0.125, 'svm__kernel': 'poly'} r (CV score=0.764):
# 
svm_clf_linear_sigmoid_t = SVM(model_name='svm_wip_linear_sigmoid', dataset_name='twitter', best_params=best_params)
svm_clf_linear_sigmoid_t.train(X_train, y_train,'linear_sigmoid')
y_pred = svm_clf_linear_sigmoid_t.predict(X_test)
svm_clf_linear_sigmoid_t.evaluate_model(y_test=y_test, y_pred=y_pred)


In [ ]:
svm_best_params = svm_clf_linear_sigmoid_t.get_best_params()
print(svm_best_params) 
svm_clf_linear_sigmoid_t_model = svm_clf_linear_sigmoid_t.get_model()

# timing_df = make_timing_curve(X_train, y_train, X_test, y_test, svm_clf_linear_sigmoid_t_model, 'svm_sigmoid_linear_1', 'twitter',
#                               'SVM (Linear)')
#{'svm__degree': 1, 'svm__kernel': 'rbf'}

In [6]:
best_params = {'dnn__batch_size': 100, 'dnn__epochs': 40, 'dnn__layer_sizes': [50, 5], 'dnn__learning_rate': 0.05, 'dnn__loss': 'mean_squared_error'} #
#{'dnn__batch_size': 10, 'dnn__epochs': 1, 'dnn__layer_sizes': [180, 50], 'dnn__learning_rate': 0.0005, 'dnn__loss': 'mean_squared_error'} #
keras_nn_clf = NN_Keras(model_name='keras_nn_1', dataset_name='twitter', best_params=best_params)
keras_nn_clf.train(X_train, y_train)
y_pred = keras_nn_clf.predict(X_test)
keras_nn_clf.evaluate_model(y_test=y_test, y_pred=y_pred)


ValueError: could not convert string to float: '20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20,20'

In [9]:
keras_nn_best_params = keras_nn_clf.get_best_params()
print(keras_nn_best_params)
keras_nn_clf_model_t = keras_nn_clf.get_model()

timing_df = make_timing_curve(X_train, y_train, X_test, y_test, keras_nn_clf_model_t, 'keras_nn_1', 'twitter',
                              'Feed-Forward Neural Network')

{'dnn__batch_size': 100, 'dnn__epochs': 3, 'dnn__layer_sizes': [50, 5], 'dnn__learning_rate': 0.05, 'dnn__loss': 'binary_crossentropy'}


Epoch 1/3


  100/33199 [..............................] - ETA: 1:06:23 - loss: 0.6786 - acc: 0.4800

  900/33199 [..............................] - ETA: 7:14 - loss: 0.6113 - acc: 0.6967   

 1500/33199 [>.............................] - ETA: 4:16 - loss: 0.5818 - acc: 0.7313

...

30300/33199 [==========================>...] - ETA: 0s - loss: 0.3110 - acc: 0.8752

31600/33199 [===========================>..] - ETA: 0s - loss: 0.3117 - acc: 0.8751

32500/33199 [============================>.] - ETA: 0s - loss: 0.3112 - acc: 0.8750

33199/33199 [==============================] - 2s 54us/step - loss: 0.3122 - acc: 0.8749


keras_nn_1 twitter 0.9
